# 01장. 텐서플로 2.0으로 신경망 구현
## 텐서플로(TensorFlow, TF)란?
- 구글 브레인 팀에서 심층 신경망(deep neural networks)을 위해 개발한 오픈소스 SW 라이브러리
- 주요 특징
	- Python, C++, Java, R, Go로 작업 가능
	- Keras는 TensorFlow와 통합된 고급 신경망 API, SW 구성 요소의 상호작용 방식 지정
	- 모델 배치와 생산 과정에서 쉽게 사용 가능
	- TensorFlow 2.0에는 정적 그래프에 기반을 둔 그래프 연산과 즉시 연산(eager computation) 지원 도입
	- 강력한 커뮤니티의 지원: Github, Google Trends에서 인기를 확인할 수 있음
    
## 케라스(Keras)란?
- 딥러닝 모델을 만들고 훈련하고자 기초 구성 요소를 구성하는 API
- 여러 딥러닝 엔진(Google의 TensorFlow, MS의 CNTK, Amazon의 MxNet, Theano etc.)
- TensorFlow 2.0부터 케라스가 표준 하이레벨 API로 채택 -> 코딩 단순화, 좀 더 직관적인 프로그래밍

## TensorFlow 2.0의 가장 중요한 변화
- Keras는 TensorFlow의 일부이기 때문에 분리에 의미가 없음
- keras vs. tf.keras
	- tf.keras는 Tensorflow 내부에 케라스 구현한 것
	- 다른 TensorFlow API와 더 나은 통합을 원하면 tf.keras 사용
- TensorFlow 1.0 설치하기
    - CPU만 있을 경우: `pip install tensorflow`
    - GPU도 있는 경우: `pip install tensorflow-gpu`
- TensorFlow 2.0 설치하기
    - CPU만 있을 경우: `pip install tensorflow==2.0.0-alpha0`
    - GPU도 있는 경우: `pip install tensorflow-gpu==2.0.0-alpha0`

In [1]:
# TensorFlow 1.0에서 신경망을 코딩하는 전통적인 방법(11 lines)
import tensorflow.compat.v1 as tf

tf.compat.v1.disable_eager_execution() # To avoid RuntimeError: tf.placeholder() is not compatible with eager execution

in_a = tf.placeholder(dtype=tf.float32, shape=(2))

def model(x):
    with tf.variable_scope("matmul"):
        W = tf.get_variable("W", initializer=tf.ones(shape=(2, 2)))
        b = tf.get_variable("b", initializer=tf.zeros(shape=(2)))
        return x * W + b

out_a = model(in_a)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    outs = sess.run([out_a], feed_dict={in_a: [1, 0]})

In [2]:
# TensorFlow 2.0으로 코딩(8 lines)
import tensorflow as tf
W = tf.Variable(tf.ones(shape=(2, 2)), name="W")
b = tf.Variable(tf.zeros(shape=(2)), name="b")

@tf.function
def model(x):
    return W * x + b

out_a = model([1, 0])

print(out_a)

Tensor("StatefulPartitionedCall:0", shape=(2, 2), dtype=float32)


## 신경망 소개: 인공 신경망(Artificial Neural Networks, nets, ANN)
- 포유류의 중추 신경계 연구에서 영감 받은 머신러닝 모델의 일종
- 각 신경망은 서로 연결된 많은 뉴런으로 구성
- 한 계층(layer)의 뉴런은 특정 상태가 되면 다른 계층으로 메시지를 교환(발화, fire)하고, 신경망은 계산 수행
- 변천 과정
    - 1950년대 후반: 단순 계산을 위한 두 계층의 퍼셉트론
    - 1960년대: 다계층 훈련을 위한 역전파(backpropagation) 알고리즘
    - 1980년대 다른 단순 기법들이 더 효과적인 방법이 될 때까지 학문 연구의 주요 주제
    - 2000년대 중반: G. Hinton이 제안한 획기적인 빠른 학습 알고리즘과 2011년경 대량 연산이 가능한 GPU의 등장과 훈련에 쓰일 대규모 데이터의 수집 가능 -> deep learning의 발판
- 점진적 추상화를 통한 학습은 인간의 두뇌에서 수백만 년 동안 진화해 온 모델과 닮았다.
    - 인간의 시각 시스템은 여러 계층으로 구성
    - 눈은 시각 피질(Visual Cortex)(V1) 영역과 연결
        - 기본적인 특징과 시각적 방향, 공간 주파수, 색상의 작은 변화 구분
        - 약 1억 4천만 개의 뉴런으로 구성, 수십억 개의 연결로 구성
        - 형태, 얼굴, 동물 등과 같이 좀 더 복잡한 개념을 인식하고 더 복잡한 이미지 처리를 하는 다른 영역(V2, V3, V4, V5, V6)와 연결
    - 딥러닝은 인간의 시각 시스템 조직에서 영감을 얻음

## 퍼셉트론(Perceptron)
- 입력 특징(feature) 또는 간단한 특징인 n개의 크기를 갖는 입력 벡터(x1, x2, ..., xn)이 주어지면 1(True) 또는 0(False)를 출력하는 간단한 알고리즘

![image](https://user-images.githubusercontent.com/61455647/117759330-91570000-b25e-11eb-9dc7-a836525c3d0c.png)

- w: 가중치 벡터, wx: dot product, b: 편향(bias)
- wx + b: w와 b에 할당된 값에 따라 위치를 변경하는 초평면(hyperplane) 경계 정의
    - 초평면: 둘러싼 공간(ambient space)보다 한 차원이 낮은 부공간(subspace)
    - 입력이 3개의 특징(빨강, 녹색, 파란색의 양)일 때 퍼셉트론은 색상이 흰색인지 아닌지를 결정
![image](https://user-images.githubusercontent.com/61455647/117759499-daa74f80-b25e-11eb-83a2-d65e3c8b50fa.png)

- 퍼셉트론은 '아마도'라는 결과를 표현할 수 없다

### TensorFlow 2.0 코드 첫 번째 예제
- tf.keras로 모델을 작성하는 방법: Sequential API, Functional API, Model Subclassing
- `Sequential()` 모델
    - 신경망 계층의 선형 파이프라인(=stack)
    - 아래의 코드는 784개의 입력 변수(feature)를 취하는 10개의 인공 뉴런을 가진 단일 계층 정의
    - 망이 밀집(dense) = 각 계층의 뉴런이 이전 계층에 위한 모든 뉴런과 완전 연결되어 있고, 그 다음 계층에 있는 모든 뉴런과도 완전 연결되어 있다.
    - 각 뉴런은 `kernel_initializer` 매개변수를 통해 특정 가중치로 초기화할 수 있다.
        - `random_uniform`: 가중치는 -0.05~0.05 사이에서 균등하게 랜덤 분포
        - `random_normal`: 가중치는 가우스 분포에 따라 평균이 0이고 작은 표준 편차 0.05로 초기화
        - `zero`: 모든 가중치는 0으로 초기화

In [3]:
import tensorflow as tf
from tensorflow import keras
NB_CLASSES = 10
RESHAPED = 784
model = tf.keras.models.Sequential()
model.add(keras.layers.Dense(NB_CLASSES, input_shape=(RESHAPED,), kernel_initializer='zeros', name='dense_layer', activation='softmax'))

## 다층 퍼셉트론: 신경망 첫 번째 예제
- Perceptron은 단일 선형 계층 모델의 이름, 단순한 선형 함수
- **다층 퍼셉트론 MLP, Multi-Layer Perceptron**
    - 여러 개의 계층이 있는 경우, 여러 개의 단일 계층이 쌓임
    - 입력과 출력 계층은 외부에서 볼 수 있지만 중간의 다른 모든 계층은 숨겨져 있음 -> **은닉층 hidden layers**
    - 첫 번째 은닉층의 각 노드는 입력 받고 선형 함수에 연계된 값에 따라 발화(fire) -> 첫 번째 은닉층의 출력은 다른 선형 함수가 적용된 두 번째 계층으로 전달 -> 하나의 단일 뉴런으로 구성된 최종 출력 계층으로 전달
    
![image](https://user-images.githubusercontent.com/61455647/117767999-ac307100-b26c-11eb-9875-5f3ab2391de6.png)

### 퍼셉트론 훈련의 문제점과 해결책
- 단일 뉴런일 때 가중치 w와 편향 b의 값으로 가장 적합한 것: 이상적으로 일련의 훈련 예를 제공하고 컴퓨터가 출력에서 발생하는 오차를 최소화하는 방식으로 가중치와 편향 조정
- ex. 고양이 이미지를 포함한 것과 그렇지 않은 별도의 이미지 집합이 있다고 가정, 각 뉴런은 이미지의 단일 픽셀 값에서 입력을 받는다고 가정
- -> 컴퓨터가 이미지를 처리하면서 각 뉴런이 가중치와 편향을 조정해 잘못 인식되는 이미지의 비율이 점차 줄어들기를 원함
- -> 출력에 아주 작은 변화만 일으키려면 가중치(or 편향)도 약간만 변경해야 함, 출력에 큰 변화가 생긴다면 점진적인 학습 X
- 퍼셉트론은 0과 1이기 때문에 '조금씩'의 작동을 보이지 않음
![image](https://user-images.githubusercontent.com/61455647/117768062-c9653f80-b26c-11eb-976f-6b672f59b7a6.png)

- -> 불연속(discontinutiy) 없이 0에서 1로 점진적으로 변경되는 함수 필요 <=> 미분 가능한 연속 함수가 필요

### 활성화 함수: 시그모이드(sigmoid)
- 𝜎(𝑥)= 1/(1+𝑒^(−𝑥))
- 입력이 (−∞, ∞)에서 변할 때 출력은 (0, 1)에서 변화

![image](https://user-images.githubusercontent.com/61455647/117768191-fdd8fb80-b26c-11eb-876f-3dc97f54dc0e.png)

- 비선형 함수 𝜎(z = w𝑥 + b) 계산에 시그모이드를 사용할 수 있다.
    - z = w𝑥 + b가 매우 크고 양수, 𝑒^(−z) -> 0이므로 𝜎(z) -> 1
    - z = w𝑥 + b가 매우 크고 음수, 𝑒^(−z) -> ∞이므로 𝜎(z) -> 0
- 시그모이드 활성화 함수를 사용한 뉴런의 경우 퍼셉트론과 유사한 작동을 하지만, 그 변화는 점진적이고 출력값도 완전히 유효하다.

### 활성화 함수: tanh
- tanh(𝑧)=(𝑒^𝑧−𝑒^(−𝑧))/(𝑒^𝑧+𝑒^(−𝑧))
- 출력 범위: [-1, 1]

![image](https://user-images.githubusercontent.com/61455647/117768839-e77f6f80-b26d-11eb-9ec6-8bd8f253cce4.png)

### 활성화 함수: ReLU(Rectified Linear Unit)
- 시그모이드에서 발견된 일부 최적화 문제를 해결하는 데에 도움
- f(x) = max(0, x)
    - 음수 값에 대해서는 항상 0, 양의 값에 대해 선형으로 증가
- 시그모이드에 비해 구현이 매우 간단

![image](https://user-images.githubusercontent.com/61455647/117769160-4b099d00-b26e-11eb-847e-19f163ca5c46.png)

### 추가적인 두 개의 활성화 함수: ELU와 LeakyReLU
- ELU

![image](https://user-images.githubusercontent.com/61455647/117770589-10a0ff80-b270-11eb-9974-0afcca168f98.png)

![image](https://user-images.githubusercontent.com/61455647/117769881-38dc2e80-b26f-11eb-9fec-5712939b3c60.png)

- LeakyReLU

![image](https://user-images.githubusercontent.com/61455647/117770387-cfa8eb00-b26f-11eb-9fa2-92d9e5b9d4af.png)

![image](https://user-images.githubusercontent.com/61455647/117770133-8b1d4f80-b26f-11eb-9798-88d712508eb3.png)

- 두 함수 모두 x가 음수일 때 작은 변화를 일으켜 경우에 따라 유용할 수 있다.

### 활성화 함수(activation functions)
- ('그래디언트 하강' 절에서) 시그모이드와 ReLU 함수가 보여주는 전형적인 점진적인 변화 형태가 신경망에서 오차를 조금씩 줄이며 적응해 나가는 학습 알고리즘을 개발하는 기본 구성 요소임을 알 수 있을 것이다.
- 입력 벡터(x1, x2, ..., xm), 가중치 벡터(w1, w2, ..., wm), 편향 b, 합계 Σ일 때 활성화 함수 𝜎는 다음과 같다.

![image](https://user-images.githubusercontent.com/61455647/117771097-b5bbd800-b270-11eb-8a9b-2fc62d26d68b.png)

### 간단히 말해: 결국 신경망이란?
- 어떤 입력을 해당 출력으로 매핑하는 함수를 계산하는 방법
- 비선형 활성화와 결합해 여러 계층으로 쌓을 경우 거의 모든 것을 학습할 수 있다.
- 최적화하려는 적절한 척도(손실함수(loss function)), 학습하기에 충분한 데이터, 충분한 연산 능력 필요
- **학습**: 본질적으로 미래의 결과를 예측하고자 확립된 관찰을 일반화하려는 것을 목표로하는 과정

## 실제 예제: 필기체 숫자 인식
- MNIST 필기체 숫자 데이터베이스 사용
- **지도학습 Supervised Learning**
    - 머신러닝에서 정답이 있는 데이터셋 사용 -> 신경망을 개선하고자 훈련 예시 사용
    - 레이블이 없다고 신경망이 예측을 수행한 후 레이블을 확인해 신경망이 얼마나 숫자를 잘 인식하는지 평가할 수 있다.

### 원핫 인코딩(OHE, One-Hot Encoding)
- 신경망 내부에 사용될 정보를 인코딩하는 간단한 도구
- 범주형 특징을 숫자형 변수로 변환
- ex. [0-9]의 값 d를 갖는 범주형 특징 수는 10개의 위치를 가진 이진 벡터로 구성해 d번째 위치만 1로 하고 나머지는 항상 0 값을 갖도록 인코딩
- 학습 알고리즘이 수치형 함수를 처리하도록 특화될 경우 사용됨

### TensorFlow 2.0으로 단순 신경망 정의
- TensorFlow 2.0은 데이터셋을 로드하고 신경망을 미세 조정하는 훈련 집합 `X_train`으로의 분할, 신경망의 성능을 평가하는 데에 사용하는 테스트 집합 `X_test`로 분할하는 적절한 라이브러리 제공
- 데이터는 신경망을 훈련할 때 32비트 정밀도를 갖도록 `float32`로 변환, [0, 1] 범위로 정규화
- 실제 레이블을 각각 `Y_train`과 `Y_test`에 로드하고 원핫 인코딩 수행
- `EPOCH`: 훈련을 얼마나 지속할 것인지
- `BATCH_SIZE`: 한 번에 신경망에 입력하는 표본의 수
- `VALIDATION`: 훈련 프로세스의 유효성 확인하거나 증명을 위해 남겨둔 데이터의 양

In [4]:
import tensorflow as tf
import numpy as np
from tensorflow import keras

# 신경망과 훈련 매개변수
EPOCHS = 200
BATCH_SIZE = 128
VERBOSE = 1
NB_CLASSES = 10  # 출력 개수 = 숫자의 개수
N_HIDDEN = 128
VALIDATION_SPLIT = 0.2 # 검증을 위해 남겨둔 훈련 데이터

# MNIST 데이터셋 로드
# 검증
# 훈련과 테스트 데이터를 각각 60000개와 10000개로 나눴다.
# 레이블에 대한 원핫 인코딩은 자동으로 적용된다.
mnist = keras.datasets.mnist
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

# X_train은 60000개 행으로 28*28 값을 가진다. -> 60000 * 784 형태로 변환
RESHAPED = 784

X_train = X_train.reshape(60000, RESHAPED)
X_test = X_test.reshape(10000, RESHAPED)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# 입력을 [0, 1] 사이로 정규화
X_train /= 255
X_test /= 255
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# 레이블을 원핫 인코딩
Y_train = tf.keras.utils.to_categorical(Y_train, NB_CLASSES)
Y_test = tf.keras.utils.to_categorical(Y_test, NB_CLASSES)

11493376/11490434 [==============================] - 0s 0us/step
60000 train samples
10000 test samples


- 입력 계층에 이미지의 각 픽셀과 연결된 뉴런이 있으며, MNIST 이미지의 각 픽셀마다 하나씩 총 28 * 28 = 784개의 뉴런이 있음
- 대개 각 픽셀과 관련된 값은 [0, 1] 범위에서 정규화된다.
- 출력은 10자리 부류(Class) 중 하나며, 각 숫자마다 하나의 부류가 있다.
- 마지막 계층은 활성화 함수가 소프트맥스(Softmax)인 단일 뉴런으로, 시그모이드 함수를 일반화한 것이다.
    - 소프트맥스: 임의의 실수 값의 K차원 벡터를 (0, 1) 범위의 실수 값을 가진 K차원 벡터로 밀어 넣어 총합이 1이 되게 한다.

In [5]:
# 모델 구축
model = tf.keras.models.Sequential()

model.add(keras.layers.Dense(NB_CLASSES, input_shape=(RESHAPED,), name='dense_layer', activation='softmax'))

- 모델 정의 후 TensorFlow 2.0에서 실행할 수 있도록 모델 컴파일 필요
- 컴파일 중 설정 사항
    1. 최적화기(optimizer) 선택
        - optimizer: 모델을 훈련시키는 동안 가중치를 업데이트하는 데 사용되는 특정 알고리즘
    2. 목적 함수(objective function) 선택
        - objective function: optimizer가 가중치 공간을 탐색하는 데에 사용
        - = 손실 함수(loss function)이나 비용 함수(cost function)
        - 최적화 프로세는 손실 최소화 프로세스로 정의
        - MSE
            - 예측과 실제 값 사이의 평균 제곱 오차
            - M𝑆𝐸= (1/n) * ∑(i=1,n)(𝑑−𝑦)^2, d: 예측 벡터, y: n 관측치의 벡터
            - 각 예측에서 발생한 모든 오차의 평균
            - 예측이 실제 값과 멀수록 제곱 연사에 의해 더욱 뚜렷해진다.
            - 제곱을 통해 오차가 양수이든 음수이든 누적 값 증가
        - binary_crossentropy
            - 로그 손실
            - 목표가 c일 때 모델이 p로 예측한 경우: 교차 엔트로피(cross-entropy) L(p, c) = -c * ln(p)-(1-c)ln(1-p)
            - 이진 레이블 예측에 적절
        - categorical_crossentropy
            - 다부류(multiclass) 로그 손실
            - 예측 분포를 참 분포와 비교 -> 참 부류에 대한 확률=1로 설정, 나머지는 0으로 설정 -> 원핫 인코딩
            - 참 부류가 c인데 y로 예측했다면, L(c, p) = -∑(i) ci * ln(pi)
            - 출력이 참 벡터에 가까울수록 손실 ↓
            - 다부류 레이블 예측에 적합
    3. 훈련된 모델을 평가 by 척도(metrics)
        - 정확도(Accuracy): 타깃 대비 정확히 예측한 비율
        - 정밀도(Precision): positive으로 예측한 것 중 실제로 참인 것의 비율
        - 재현율(Recall): 올바르게 예측한 것(참은 positive, 거짓은 negative으로 예측) 중 positive으로 예측한 것이 실제로 참인 비율
        - 모델 평가에만 사용, 신경망의 성능 판단

In [6]:
# 모델 컴파일
model.compile(optimizer='SGD', loss='categorical_crossentropy', metrics=['accuracy'])

- 확률적 그래디언트 하강(SGD, Stochastic Gradient Descent)
    - 최적화 알고리즘의 특별한 종류
    - 각 훈련 에폭(epoch)마다 신경망의 오차를 줄이고자 사용
- 모델 컴파일 후 `fit()` 메소드로 훈련할 수 있고, 이때 매개변수 명시 가능
    - `epochs`
        - 모델이 훈련 집합에 노출된 횟수
        - 각 반복에서 optimizer는 목표 함수가 최소가 되도록 가중치를 조정
    - `batch_size`
        - optimizer가 가중치 갱신을 수행하기 전에 관찰한 훈련 인스턴스의 수
        - 한 에폭당 여러 배치가 있다.

In [7]:
# 모델 훈련
model.fit(X_train, Y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=VERBOSE, validation_split=VALIDATION_SPLIT)

Train on 48000 samples, validate on 12000 samples
Epoch 1/200
48000/48000 [==============================] - 1s 22us/sample - loss: 1.3597 - accuracy: 0.6863 - val_loss: 0.8869 - val_accuracy: 0.8321
Epoch 2/200
 3840/48000 [=>............................] - ETA: 0s - loss: 0.8951 - accuracy: 0.8266

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


48000/48000 [==============================] - 1s 16us/sample - loss: 0.7853 - accuracy: 0.8351 - val_loss: 0.6525 - val_accuracy: 0.8603
Epoch 3/200
48000/48000 [==============================] - 1s 17us/sample - loss: 0.6389 - accuracy: 0.8544 - val_loss: 0.5588 - val_accuracy: 0.8706
Epoch 4/200
48000/48000 [==============================] - 1s 17us/sample - loss: 0.5680 - accuracy: 0.8644 - val_loss: 0.5070 - val_accuracy: 0.8772
Epoch 5/200
48000/48000 [==============================] - 1s 17us/sample - loss: 0.5248 - accuracy: 0.8704 - val_loss: 0.4731 - val_accuracy: 0.8831
Epoch 6/200
48000/48000 [==============================] - 1s 17us/sample - loss: 0.4950 - accuracy: 0.8746 - val_loss: 0.4491 - val_accuracy: 0.8871
Epoch 7/200
48000/48000 [==============================] - 1s 17us/sample - loss: 0.4730 - accuracy: 0.8784 - val_loss: 0.4315 - val_accuracy: 0.8911
Epoch 8/200
48000/48000 [==============================] - 1s 17us/sample - loss: 0.4558 - accuracy: 0.8809 - va

- 훈련하는 동안 유효성 성능을 측정하고자 훈련 데이터의 일부를 남겨둔다.
- 모델 훈련 후 훈련 과정에서 모델이 한 번도 본 적 없는 새로운 예시의 테스트 집합으로 평가
- `evaluate(X_test, Y_test)`로 `test_loss`와 `test_acc` 계산

In [8]:
# 모델 평가
test_loss, test_acc = model.evaluate(X_test, Y_test)
print('Test accuracy: ', test_acc)

Test accuracy:  0.9231


### 단순 TensorFlow 2.0 신경망 실행과 베이스라인 구축
- 신경망의 아키텍처가 출력되고, 사용된 여러 계층의 유형, 출력 형태, 최적화해야 할 매개변수 개수(= 가중치 수)와 연결 방식 확인
- 신경망은 48000개의 표본으로 훈련, 12000개의 표본은 검증을 위해 사용됨
- 훈련 후에 테스트 집합에서 모델을 테스트

### TensorFlow 2.0의 단순 신경망을 은닉층으로 개선
- 개선법: 신경망에 계층 추가
    - ∵ 추가 뉴런은 훈련 데이터에서 좀 더 복잡한 패턴을 학습하는 데 도움
    - -> 계층 추가로 매개변수가 추가돼 모델이 더 복잡한 패턴을 기억할 수 있게 된다.
    - -> 변경: 입력 계층 -> N_HIDDEN 뉴런과 활성화 함수 ReLU로 첫 번째 밀집 계층 추가(= hidden) -> N_HIDDEN 뉴런을 가진 2번째 은닉층

In [9]:
import tensorflow as tf
from tensorflow import keras

# 신경망과 훈련
EPOCHS = 50
BATCH_SIZE = 128
VERBOSE = 1
NB_CLASSES = 10 # 출력 개수 = 숫자 개수
N_HIDDEN = 128
VALIDATION_SPLIT = 0.2  # 검증에 남겨둘 훈련 집합 부분

# MNIST 데이터셋 로드
# 레이블은 원핫 표기로 되어 있다.
mnist = keras.datasets.mnist
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

# X_train은 60000개 행의 28*28 값 -> 60000*784 형태로 변경
RESHAPED = 784

X_train = X_train.reshape(60000, RESHAPED)
X_test = X_test.reshape(10000, RESHAPED)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# 입력을 [0, 1] 사이로 정규화
X_train, X_test = X_train / 255.0, X_test / 255.0
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

Y_train = tf.keras.utils.to_categorical(Y_train, NB_CLASSES)
Y_test = tf.keras.utils.to_categorical(Y_test, NB_CLASSES)

# 모델 구축
model = tf.keras.models.Sequential()
model.add(keras.layers.Dense(N_HIDDEN, input_shape=(RESHAPED,), name='dense_layer', activation='relu'))
model.add(keras.layers.Dense(N_HIDDEN, name='dense_layer_2', activation='relu'))
model.add(keras.layers.Dense(NB_CLASSES, name='dense_layer_3', activation='softmax'))

# 모델 요약
model.summary()

# 모델 컴파일
model.compile(optimizer='SGD', loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 훈련
model.fit(X_train, Y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=VERBOSE, validation_split=VALIDATION_SPLIT)

# 모델 평가
test_loss, test_acc = model.evaluate(X_test, Y_test)
print('Test accuracy: ', test_acc)

60000 train samples
10000 test samples
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_layer (Dense)          (None, 128)               100480    
_________________________________________________________________
dense_layer_2 (Dense)        (None, 128)               16512     
_________________________________________________________________
dense_layer_3 (Dense)        (None, 10)                1290      
Total params: 118,282
Trainable params: 118,282
Non-trainable params: 0
_________________________________________________________________
Train on 48000 samples, validate on 12000 samples
Epoch 1/50
48000/48000 [==============================] - 1s 29us/sample - loss: 1.4358 - accuracy: 0.6379 - val_loss: 0.7136 - val_accuracy: 0.8407
Epoch 2/50
  128/48000 [..............................] - ETA: 1s - loss: 0.8927 - accuracy: 0.7656

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


48000/48000 [==============================] - 1s 27us/sample - loss: 0.5795 - accuracy: 0.8541 - val_loss: 0.4430 - val_accuracy: 0.8852
Epoch 3/50
48000/48000 [==============================] - 1s 27us/sample - loss: 0.4318 - accuracy: 0.8834 - val_loss: 0.3700 - val_accuracy: 0.8986
Epoch 4/50
48000/48000 [==============================] - 1s 27us/sample - loss: 0.3765 - accuracy: 0.8949 - val_loss: 0.3373 - val_accuracy: 0.9047
Epoch 5/50
48000/48000 [==============================] - 1s 27us/sample - loss: 0.3449 - accuracy: 0.9024 - val_loss: 0.3124 - val_accuracy: 0.9112
Epoch 6/50
48000/48000 [==============================] - 1s 27us/sample - loss: 0.3231 - accuracy: 0.9084 - val_loss: 0.2969 - val_accuracy: 0.9153
Epoch 7/50
48000/48000 [==============================] - 1s 27us/sample - loss: 0.3062 - accuracy: 0.9132 - val_loss: 0.2834 - val_accuracy: 0.9198
Epoch 8/50
48000/48000 [==============================] - 1s 27us/sample - loss: 0.2924 - accuracy: 0.9172 - val_loss

- `to_categorical(Y_train, NB_CLASSES)`: 배열 `Y_train`을 부류 개수만큼 열을 가진 행렬로 변환
- 1번째 학습 vs. 2번째 학습(은닉층 2개 추가): 테스트 정확도 ↑, 반복 횟수 200 -> 50
- **수렴 convergence**: 일정 에폭을 넘어서면 개선의 효과가 미미함

### TensorFlow에서 드롭아웃(Dropout)으로 단순망 개선
- 훈련 중에 은닉층 내부 밀집 신경망에 전파된 값 중 일부를 무작위로 제거 -> 성능 향상
- **무작위 드롭아웃 Random Dropout**: 신경망의 일반화를 향상시키는 데 도움 되는 유용한 중복 패턴 학습

In [10]:
import tensorflow as tf
import numpy as np
from tensorflow import keras

# 신경망과 훈련
EPOCHS = 200
BATCH_SIZE = 128
VEBOSE = 1
NB_CLASSES = 10  # 출력 개수 = 숫자 개수
N_HIDDEN = 128
VALIDATION_SPLIT = 0.2  # VALIDATION을 위해 예약된 TRAIN의 양
DROPOUT = 0.3

# MNIST 데이터셋 로드
# 레이블은 원핫 인코딩으로 표현
mnist = keras.datasets.mnist
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

# X_train은 60000개 행의 28*28 값 -> 60000*784 형태로 변경
RESHAPED = 784

X_train = X_train.reshape(60000, RESHAPED)
X_test = X_test.reshape(10000, RESHAPED)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# 입력을 [0, 1] 사이로 정규화
X_train, X_test = X_train / 255.0, X_test / 255.0
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'teset samples')

Y_train = tf.keras.utils.to_categorical(Y_train, NB_CLASSES)
Y_test = tf.keras.utils.to_categorical(Y_test, NB_CLASSES)

# 모델 구축
model = tf.keras.models.Sequential()
model.add(keras.layers.Dense(N_HIDDEN, input_shape=(RESHAPED,), name='dense_layer', activation='relu'))
model.add(keras.layers.Dropout(DROPOUT))
model.add(keras.layers.Dense(N_HIDDEN, name='dense_layer_2', activation='relu'))
model.add(keras.layers.Dropout(DROPOUT))
model.add(keras.layers.Dense(NB_CLASSES, name='dense_layer_3', activation='softmax'))

# 모델 요약
model.summary()

# 모델 컴파일
model.compile(optimizer='SGD', loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 훈련
model.fit(X_train, Y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=VERBOSE, validation_split=VALIDATION_SPLIT)

# 모델 평가
test_loss, test_acc = model.evaluate(X_test, Y_test)
print('Test accuracy: ', test_acc)

60000 train samples
10000 teset samples
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_layer (Dense)          (None, 128)               100480    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_layer_2 (Dense)        (None, 128)               16512     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_layer_3 (Dense)        (None, 10)                1290      
Total params: 118,282
Trainable params: 118,282
Non-trainable params: 0
_________________________________________________________________
Train on 48000 samples, validate on 12000 samples
Epoch 1/200
48000/48000 

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


Epoch 2/200
48000/48000 [==============================] - 2s 36us/sample - loss: 0.9110 - accuracy: 0.7175 - val_loss: 0.5224 - val_accuracy: 0.8698
Epoch 3/200
48000/48000 [==============================] - 2s 35us/sample - loss: 0.6921 - accuracy: 0.7885 - val_loss: 0.4122 - val_accuracy: 0.8927
Epoch 4/200
48000/48000 [==============================] - 2s 36us/sample - loss: 0.5934 - accuracy: 0.8190 - val_loss: 0.3643 - val_accuracy: 0.9025
Epoch 5/200
48000/48000 [==============================] - 2s 36us/sample - loss: 0.5299 - accuracy: 0.8396 - val_loss: 0.3309 - val_accuracy: 0.9090
Epoch 6/200
48000/48000 [==============================] - 2s 36us/sample - loss: 0.4858 - accuracy: 0.8556 - val_loss: 0.3089 - val_accuracy: 0.9141
Epoch 7/200
48000/48000 [==============================] - 2s 36us/sample - loss: 0.4519 - accuracy: 0.8647 - val_loss: 0.2909 - val_accuracy: 0.9171
Epoch 8/200
48000/48000 [==============================] - 2s 35us/sample - loss: 0.4302 - accuracy:

- 내부 은닉층에서 무작위로 드롭아웃하는 신경망이 테스트 집합에 포함된 낯선(unseen) 예시를 잘 '일반화'한다.
    - ∵ 각각의 뉴런이 자기 이웃에 의존할 수 없다는 것을 인식, 중복된 방식으로 정보가 저장되도록 강제
- 테스트 중에는 드롭아웃이 없으므로, 모든 뉴런이 사용된다.
- 훈련 정확도는 테스트 정확도보다 높아야 한다. 그렇지 않다면 에폭의 수를 더 늘려야 한다.

### TensorFlow 2.0에서 여러 Optimizer 테스트

![image](https://user-images.githubusercontent.com/61455647/117948802-72369c00-b34c-11eb-93ae-9d2cd74a3a09.png)

- 그래디언트 하강(Gradient Descent, GD)
    - 하나의 단일 변수 w에 대해 일반적인 비용 함수 C(w)가 있다고 가정하자.
    - 목표: 출발점 w0에서 아주 조금씩 움직이면서 경사(함수 C)면을 따라 내려가 도랑(최소 Cmin) 찾기
    - 각 단계 r에서 gradient = 최대 증가 방향 -> 단계 r에서 도달한 지점 wr에서 계산된 편미분 값 ∂C/∂w
    - => 반대 방향인 -(∂C/∂w)(wr)을 택하면 도랑을 향해 나아갈 수 있다.
    - 학습률 η(>= 0): 다음 단계의 보폭
        - η이 너무 작다면 천천히 이동
        - η이 너무 크다면 도랑을 지나칠 가능성이 있다.
    - sigmoid function
        - 연속이고 미분 가능
        - 𝜎(𝑥)= 1/(1+𝑒^(−𝑥))이면, d𝜎(𝑥)/d(x) = 𝜎(𝑥)(1-𝜎(𝑥))
    - ReLU function
        - 0에서 미분 불가능
        - -> 0에서 미분 값을 0이나 1로 임의 지정하면 전체 범위로 확장 가능
        - y = max(0, x)의 부분 미분 dy/dx = 0(x <= 0), 1(x > 0) -> GD를 통해 망을 최적화할 수 있다.
- TensorFlow에는 GD의 속도를 높이기 위한 변형인 SGD와 RMSProp, Adam 등의 최적화 기술이 제공된다.
    - RMSProp, Adam은 SGD의 가속 구성 요소 외에도 momentum(속도 요소) 개념 포함 -> 많은 계산을 통한 더 빠른 수렴
        - momentum을 통해 SGD를 유관 방향으로 가속화하고 이동하는 것을 줄일 수 있다.

In [11]:
# RMSProp, epochs = 200
import tensorflow as tf
import numpy as np
from tensorflow import keras

# 신경망과 훈련
EPOCHS = 200
BATCH_SIZE = 128
VEBOSE = 1
NB_CLASSES = 10  # 출력 개수 = 숫자 개수
N_HIDDEN = 128
VALIDATION_SPLIT = 0.2  # VALIDATION을 위해 예약된 TRAIN의 양
DROPOUT = 0.3

# MNIST 데이터셋 로드
# 레이블은 원핫 인코딩으로 표현
mnist = keras.datasets.mnist
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

# X_train은 60000개 행의 28*28 값 -> 60000*784 형태로 변경
RESHAPED = 784

X_train = X_train.reshape(60000, RESHAPED)
X_test = X_test.reshape(10000, RESHAPED)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# 입력을 [0, 1] 사이로 정규화
X_train, X_test = X_train / 255.0, X_test / 255.0
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'teset samples')

Y_train = tf.keras.utils.to_categorical(Y_train, NB_CLASSES)
Y_test = tf.keras.utils.to_categorical(Y_test, NB_CLASSES)

# 모델 구축
model = tf.keras.models.Sequential()
model.add(keras.layers.Dense(N_HIDDEN, input_shape=(RESHAPED,), name='dense_layer', activation='relu'))
model.add(keras.layers.Dropout(DROPOUT))
model.add(keras.layers.Dense(N_HIDDEN, name='dense_layer_2', activation='relu'))
model.add(keras.layers.Dropout(DROPOUT))
model.add(keras.layers.Dense(NB_CLASSES, name='dense_layer_3', activation='softmax'))

# 모델 요약
model.summary()

# 모델 컴파일
# 변경 부분
model.compile(optimizer='RMSProp', loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 훈련
model.fit(X_train, Y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=VERBOSE, validation_split=VALIDATION_SPLIT)

# 모델 평가
test_loss, test_acc = model.evaluate(X_test, Y_test)
print('Test accuracy: ', test_acc)

60000 train samples
10000 teset samples
Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_layer (Dense)          (None, 128)               100480    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_layer_2 (Dense)        (None, 128)               16512     
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_layer_3 (Dense)        (None, 10)                1290      
Total params: 118,282
Trainable params: 118,282
Non-trainable params: 0
_________________________________________________________________
Train on 48000 samples, validate on 12000 samples
Epoch 1/200
47616/48000 

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


48000/48000 [==============================] - 2s 46us/sample - loss: 0.4723 - accuracy: 0.8575 - val_loss: 0.1786 - val_accuracy: 0.9469
Epoch 2/200
48000/48000 [==============================] - 2s 42us/sample - loss: 0.2224 - accuracy: 0.9338 - val_loss: 0.1324 - val_accuracy: 0.9599
Epoch 3/200
48000/48000 [==============================] - 2s 42us/sample - loss: 0.1737 - accuracy: 0.9481 - val_loss: 0.1204 - val_accuracy: 0.9639
Epoch 4/200
48000/48000 [==============================] - 2s 43us/sample - loss: 0.1505 - accuracy: 0.9559 - val_loss: 0.1072 - val_accuracy: 0.9688
Epoch 5/200
48000/48000 [==============================] - 2s 42us/sample - loss: 0.1321 - accuracy: 0.9600 - val_loss: 0.1118 - val_accuracy: 0.9670
Epoch 6/200
48000/48000 [==============================] - 2s 42us/sample - loss: 0.1254 - accuracy: 0.9624 - val_loss: 0.1066 - val_accuracy: 0.9718
Epoch 7/200
48000/48000 [==============================] - 2s 44us/sample - loss: 0.1102 - accuracy: 0.9681 - va

In [12]:
# RMSProp, epochs = 250
import tensorflow as tf
import numpy as np
from tensorflow import keras

# 신경망과 훈련
EPOCHS = 250
BATCH_SIZE = 128
VEBOSE = 1
NB_CLASSES = 10  # 출력 개수 = 숫자 개수
N_HIDDEN = 128
VALIDATION_SPLIT = 0.2  # VALIDATION을 위해 예약된 TRAIN의 양
DROPOUT = 0.3

# MNIST 데이터셋 로드
# 레이블은 원핫 인코딩으로 표현
mnist = keras.datasets.mnist
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

# X_train은 60000개 행의 28*28 값 -> 60000*784 형태로 변경
RESHAPED = 784

X_train = X_train.reshape(60000, RESHAPED)
X_test = X_test.reshape(10000, RESHAPED)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# 입력을 [0, 1] 사이로 정규화
X_train, X_test = X_train / 255.0, X_test / 255.0
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'teset samples')

Y_train = tf.keras.utils.to_categorical(Y_train, NB_CLASSES)
Y_test = tf.keras.utils.to_categorical(Y_test, NB_CLASSES)

# 모델 구축
model = tf.keras.models.Sequential()
model.add(keras.layers.Dense(N_HIDDEN, input_shape=(RESHAPED,), name='dense_layer', activation='relu'))
model.add(keras.layers.Dropout(DROPOUT))
model.add(keras.layers.Dense(N_HIDDEN, name='dense_layer_2', activation='relu'))
model.add(keras.layers.Dropout(DROPOUT))
model.add(keras.layers.Dense(NB_CLASSES, name='dense_layer_3', activation='softmax'))

# 모델 요약
model.summary()

# 모델 컴파일
# 변경 부분
model.compile(optimizer='RMSProp', loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 훈련
model.fit(X_train, Y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=VERBOSE, validation_split=VALIDATION_SPLIT)

# 모델 평가
test_loss, test_acc = model.evaluate(X_test, Y_test)
print('Test accuracy: ', test_acc)

60000 train samples
10000 teset samples
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_layer (Dense)          (None, 128)               100480    
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_layer_2 (Dense)        (None, 128)               16512     
_________________________________________________________________
dropout_5 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_layer_3 (Dense)        (None, 10)                1290      
Total params: 118,282
Trainable params: 118,282
Non-trainable params: 0
_________________________________________________________________
Train on 48000 samples, validate on 12000 samples
Epoch 1/250
47104/48000 

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


48000/48000 [==============================] - 2s 48us/sample - loss: 0.4700 - accuracy: 0.8581 - val_loss: 0.1716 - val_accuracy: 0.9508
Epoch 2/250
48000/48000 [==============================] - 2s 44us/sample - loss: 0.2192 - accuracy: 0.9351 - val_loss: 0.1384 - val_accuracy: 0.9590
Epoch 3/250
48000/48000 [==============================] - 2s 45us/sample - loss: 0.1678 - accuracy: 0.9506 - val_loss: 0.1162 - val_accuracy: 0.9668
Epoch 4/250
48000/48000 [==============================] - 2s 44us/sample - loss: 0.1441 - accuracy: 0.9578 - val_loss: 0.1024 - val_accuracy: 0.9706
Epoch 5/250
48000/48000 [==============================] - 2s 44us/sample - loss: 0.1266 - accuracy: 0.9621 - val_loss: 0.1021 - val_accuracy: 0.9722
Epoch 6/250
48000/48000 [==============================] - 2s 43us/sample - loss: 0.1164 - accuracy: 0.9653 - val_loss: 0.0982 - val_accuracy: 0.9712
Epoch 7/250
48000/48000 [==============================] - 2s 44us/sample - loss: 0.1077 - accuracy: 0.9672 - va

- RMSDrop을 이용할 때 에폭 수가 증가할 때 훈련과 테스트 집합에서 정확도가 어떻게 증가하는지는 다음과 같다.
- -> 약 15에폭에서 서로 맞닿고 그 이후에는 훈련할 필요가 없다.

![image](https://user-images.githubusercontent.com/61455647/117957768-58e61d80-b355-11eb-9ee2-9bc8bca20594.png)


In [13]:
# Adam, epochs = 200
import tensorflow as tf
import numpy as np
from tensorflow import keras

# 신경망과 훈련
EPOCHS = 200
BATCH_SIZE = 128
VEBOSE = 1
NB_CLASSES = 10  # 출력 개수 = 숫자 개수
N_HIDDEN = 128
VALIDATION_SPLIT = 0.2  # VALIDATION을 위해 예약된 TRAIN의 양
DROPOUT = 0.3

# MNIST 데이터셋 로드
# 레이블은 원핫 인코딩으로 표현
mnist = keras.datasets.mnist
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

# X_train은 60000개 행의 28*28 값 -> 60000*784 형태로 변경
RESHAPED = 784

X_train = X_train.reshape(60000, RESHAPED)
X_test = X_test.reshape(10000, RESHAPED)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# 입력을 [0, 1] 사이로 정규화
X_train, X_test = X_train / 255.0, X_test / 255.0
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'teset samples')

Y_train = tf.keras.utils.to_categorical(Y_train, NB_CLASSES)
Y_test = tf.keras.utils.to_categorical(Y_test, NB_CLASSES)

# 모델 구축
model = tf.keras.models.Sequential()
model.add(keras.layers.Dense(N_HIDDEN, input_shape=(RESHAPED,), name='dense_layer', activation='relu'))
model.add(keras.layers.Dropout(DROPOUT))
model.add(keras.layers.Dense(N_HIDDEN, name='dense_layer_2', activation='relu'))
model.add(keras.layers.Dropout(DROPOUT))
model.add(keras.layers.Dense(NB_CLASSES, name='dense_layer_3', activation='softmax'))

# 모델 요약
model.summary()

# 모델 컴파일
# 변경 부분
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 훈련
model.fit(X_train, Y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, verbose=VERBOSE, validation_split=VALIDATION_SPLIT)

# 모델 평가
test_loss, test_acc = model.evaluate(X_test, Y_test)
print('Test accuracy: ', test_acc)

60000 train samples
10000 teset samples
Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_layer (Dense)          (None, 128)               100480    
_________________________________________________________________
dropout_6 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_layer_2 (Dense)        (None, 128)               16512     
_________________________________________________________________
dropout_7 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_layer_3 (Dense)        (None, 10)                1290      
Total params: 118,282
Trainable params: 118,282
Non-trainable params: 0
_________________________________________________________________
Train on 48000 samples, validate on 12000 samples
Epoch 1/200
47488/48000 

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


48000/48000 [==============================] - 2s 45us/sample - loss: 0.5183 - accuracy: 0.8411 - val_loss: 0.1805 - val_accuracy: 0.9489
Epoch 2/200
48000/48000 [==============================] - 2s 40us/sample - loss: 0.2319 - accuracy: 0.9307 - val_loss: 0.1401 - val_accuracy: 0.9603
Epoch 3/200
48000/48000 [==============================] - 2s 41us/sample - loss: 0.1802 - accuracy: 0.9454 - val_loss: 0.1153 - val_accuracy: 0.9663
Epoch 4/200
48000/48000 [==============================] - 2s 41us/sample - loss: 0.1468 - accuracy: 0.9564 - val_loss: 0.1036 - val_accuracy: 0.9672
Epoch 5/200
48000/48000 [==============================] - 2s 41us/sample - loss: 0.1286 - accuracy: 0.9615 - val_loss: 0.1025 - val_accuracy: 0.9682
Epoch 6/200
48000/48000 [==============================] - 2s 41us/sample - loss: 0.1173 - accuracy: 0.9644 - val_loss: 0.0934 - val_accuracy: 0.9725
Epoch 7/200
48000/48000 [==============================] - 2s 40us/sample - loss: 0.1081 - accuracy: 0.9673 - va

- Adam을 사용할 때 에폭 수가 증가할 때 훈련 및 테스트 집합에서 정확도의 증가는 다음과 같은 양상을 보인다.
- Adam을 optimizer로 선택한다면 약 12에폭 단계 후 그만둘 수 있다.

![image](https://user-images.githubusercontent.com/61455647/117958445-f9d4d880-b355-11eb-8310-17b0fb0faa52.png)

- 다양한 드롭아웃 값에 대해 테스트 데이터셋의 정확도는 다음과 같은 양상을 나타낸다.

![image](https://user-images.githubusercontent.com/61455647/117958546-153fe380-b356-11eb-9a4e-fac431cb454c.png)

### 에폭 수 증가시키기
- 에폭 수를 20에서 200으로 늘려도 계산 시간은 10배 증가되지만, 결과가 개선되지는 않는다.
- -> 학습에는 계산에 소요된 시간보다 적절한 기술 채택이 더 중요하다.

![image](https://user-images.githubusercontent.com/61455647/117958817-5c2dd900-b356-11eb-9357-385833d6f86e.png)

### Optimizer 학습률 조절
- 서로 다른 학습률에 따른 정확도는 다음과 같다. lr=0.1일 때 최상의 값이므로 이 값이 기본 학습 속도이다.

![image](https://user-images.githubusercontent.com/61455647/117959689-39e88b00-b357-11eb-8c1c-6345c94a0e45.png)

### 내부 은닉층 개수 증가
- 은닉 뉴런의 수가 증가함에 따라 모델의 복잡도가 증가해, 최적화해야 할 매개변수가 점점 많아서 실행 시간이 크게 증가한다.
- 신경망의 크기를 증가시켜 얻는 이득은 망이 증가함에 따라 점점 감소한다.
- ∴ 은닉 뉴런의 수를 일정 이상 증가시키면 신경망이 일반화가 어려워져 정확도가 저하될 수 있다.

![image](https://user-images.githubusercontent.com/61455647/117960101-9fd51280-b357-11eb-9258-153ad56d2d23.png)

![image](https://user-images.githubusercontent.com/61455647/117960263-c98e3980-b357-11eb-9792-a42ec88b49b8.png)

### 배치 계산 크기 증가
- 자료를 통해 BATCH_SIZE=64일 때 최고 정확도에 도달함을 알 수 있다.

![image](https://user-images.githubusercontent.com/61455647/117963701-ab2a3d00-b35b-11eb-8c28-fd1ceb06ee7f.png)

### 필기체 인식 실행 차트 요약
- 변형을 통해 성능을 향상시킬 수 있었다.
    1. TensorFlow 2.0으로 단일 계층 신경망 정의
    2. 은닉층 추가
    3. 신경망에 임의의 드롭아웃 추가
    4. RMSProp과 Adam으로 테스트 집합의 성능 개선
    
|모델/정확도|훈련|검증|테스트
|-|-|-|-|
|단일|89.96%|90.70%|90.71%|
|2 은닉(128)|90.81%|91.40%|91.18%|
|드롭아웃(30%)|91.70%|94.42%|94.15%(200 에폭)|
|RMSProp|97.43%|97.62%|97.64%(10 에폭)|
|Adam|98.94%|97.89%|97.82(10 에폭)|

- 다음 두 실험은 큰 개선점이 없었다.
    1. 내부 뉴런의 수를 늘리면 더 복잡한 모델이 생성되고 더 많은 계산량이 필요하지만 개선의 정도가 미미하다.
    2. optimizer의 BATCH_SIZE를 변경해도 개선의 정도가 미미하다.

## 정규화
### 과적합을 피하기 위한 정규화 적용
- 좋은 머신러닝 모델 = 훈련 데이터에서 낮은 오류율 <=> 주어진 훈련 데이터에 대해 모델의 손실 함수 최소화 -> min: {손실(훈련 데이터|모델)}
- 훈련 데이터에 내재된 모든 관계를 포착하려다 모델의 복잡도 증가
    - 부정적인 결과 1. 복잡한 모델 실행에 상당한 시간 소요
    - 부정적인 결과 2. 훈련 데이터에서 우수한 성과를 달성할 수 있지만, 검증 데이터에서 나쁜 성과를 보일 수 있다.
        - ∵ 모델이 훈련에만 특화된 많은 매개변수를 고려할 순 있지만, 이는 일반적이지 않음
        - -> **과적합 overfitting**: 일반화 능력을 잃은 모델
        
![image](https://user-images.githubusercontent.com/61455647/117966880-64d6dd00-b35f-11eb-9613-096fa71e8b0b.png)

- 훈련 과정에서 초기 감소 후 검증 단계에서 손실이 증가한다면, 모델의 복잡성 문제가 발생한 것이다. -> 훈련 데이터 과적합
- 과적합 문제를 해결하기 위해 모델의 복잡도를 파악할 수 있어야 한다.
    - 모델 = 가중치의 벡터
    - 각 가중치는 0이나 0에 매우 가깝지 않으면 출력에 영향을 준다.
    - ∴ 모델의 복잡도는 0이 아닌 가중치의 개수로 표현될 수 있다. <=> 손실 함수 측면에서 0이 아닌 가중치의 개수가 최소인 가장 간단한 모델 선택
    - 초매개변수 λ >= 0으로 단순 모델의 중요성 조절 -> min: {손실(훈련 데이터|모델)} + λ * 복잡도(모델)
- 정규화 방법
    - L1 정규화(LASSO): 모델의 복잡도는 가중치 절댓값의 합
    - L2 정규화(Ridge): 모델의 복잡도는 가중치 제곱의 합
    - Elastic 정규화: 모델의 복잡도는 L1, L2 정규화의 조합
- 정규화로 과적합이 분명할 때 신경망의 성능을 향상시킬 수 있다.
- TensorFlow는 L1, L2, ElasticNet 정류화를 지원한다.
```
from tf.keras.regularizers import l2, activity_l2
model.add(Dense(64, input_dim=64, W_regularizer=l2(0,01), activity_regularizer=activity_l2(0.01)))
```

### 배치 정규화(Batch Normalization)의 이해
- 경우에 따라 훈련 에폭을 절반으로 줄여 훈련을 가속화할 수 있다.
- 기존의 문제점
    1. 각 계층은 모든 배치마다 가중치를 지속적으로 다른 분포로 조정한다. -> 모델 훈련 속도 ↓ => 각 배치와 각 에폭에 대해 계층 입력이 좀 더 유사한 분포를 갖도록 하자.
    2. 시그모이드 함수는 값이 0에서 상당히 멀어지면 고착되어 가중치가 갱신되지 않는다. => 계층 출력을 0에 가까운 Gaussian 분포 단위로 변환
- 해결책
    - 활성화 입력 x, 배치 평균 μ, 배치 분산 σ,  작은 수 ε, 선형 변환 y = λx + β
    - (x-μ)로 0 주위로 모음 -> (x-μ)/(σ+ε)로 분모가 0이 되는 것을 피함 -> y = λx + β로 훈련 단계에서 정규화 효과 적용
    - -> 다른 계층에서도 훈련 과정에서 λ와 β 매개변수 최적화
- 활성화 속도가 너무 작아 없어지거나 너무 커지는 것을 방지하는 데에 도움 -> 훈련 속도와 정확도 ↑

## 구글 Colab 사용: CPU, GPU, TPU

## 감정 분석(IMDb 데이터셋으로 개발된 감정 분석 예시)
- IMDb 데이터셋: Internet Movie Database의 50000개의 영화 리뷰 텍스트(긍정/부정) -> 25000건의 훈련과 25000건의 테스트 집합으로 나눈다.
- 목표: 텍스트로 이진 판단을 예측할 수 있는 분류기 구축
- 데이터셋 설명
  - `tf.keras`로 IMDb 로드
  - 리뷰의 단어 시퀀스는 정수의 시퀀스로 변환 <=> 각 정수 = 사전의 특정 단어
  - 문장을 `max_len` 길이로 채워 길이에 상관 없이 모든 문장을 신경망에 입력
  - 각 입력 벡터는 고정된 크기

In [14]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, preprocessing
import tensorflow_datasets as tfds

max_len = 200
n_words = 10000
dim_embedding = 256
EPOCHS = 20
BATCH_SIZE = 500

def load_data():
  # 데이터 로드
  (X_train, y_train), (X_test, y_test) = datasets.imdb.load_data(num_words=n_words)
  # 문장을 max_len이 되도록 채워 넣는다.
  X_train = preprocessing.sequence.pad_sequences(X_train, maxlen=max_len)
  X_test = preprocessing.sequence.pad_sequences(X_test, maxlen=max_len)
  return (X_train, y_train), (X_test, y_test)

- 모델 설명
  - `Embedding()` 계층으로 리뷰에 포함된 단어의 희소 공간을 더 조밀한 공간으로 매핑 -> 계산이 용이
  - `GlobalMaxPooling1D()` 계층으로 `n_words`의 특징 벡터의 최댓값을 얻음
  - 2개의 `Dense()` 계층
  - 마지막은 단일 뉴런: 최종 이진 추정을 위해 시그모이드 활성화 함수

In [15]:
def build_model():
  model = models.Sequential()
  # 입력: -eEmbedding Layer
  # 모델의 입력: 크기의 정수 행렬(batch, input_length), 모델의 출력: 차원(input_length, dim_embedding)
  # 입력 중 가장 큰 정수는 n_words보다 작거나 같다.
  model.add(layers.Embedding(n_words, dim_embedding, input_length=max_len))

  model.add(layers.Dropout(0.3))

  # 각 n_words 특징에서 특징 벡터의 최대를 취함
  model.add(layers.GlobalMaxPooling1D())
  model.add(layers.Dense(128, activation='relu'))
  model.add(layers.Dropout(0.5))
  model.add(layers.Dense(1, activation='sigmoid'))
  return model


# 모델 훈련
(X_train, y_train), (X_test, y_test) = load_data()
model = build_model()
model.summary()

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

score = model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(X_test, y_test))

score = model.evaluate(X_test, y_test, batch_size=BATCH_SIZE)
print("\nTest score: ", score[0])
print('Test accuracy: ', score[1])

17465344/17464789 [==============================] - 0s 0us/step


<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 200, 256)          2560000   
_________________________________________________________________
dropout_8 (Dropout)          (None, 200, 256)          0         
_________________________________________________________________
global_max_pooling1d (Global (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               32896     
_________________________________________________________________
dropout_9 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 2,593,025
Trainable params: 2,593,025
Non-trainable params: 0
____________________________________________

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


25000/25000 [==============================] - 24s 962us/sample - loss: 0.6717 - accuracy: 0.6377 - val_loss: 0.6301 - val_accuracy: 0.8304
Epoch 2/20
25000/25000 [==============================] - 24s 951us/sample - loss: 0.4586 - accuracy: 0.8401 - val_loss: 0.3617 - val_accuracy: 0.8583
Epoch 3/20
25000/25000 [==============================] - 24s 951us/sample - loss: 0.2806 - accuracy: 0.8868 - val_loss: 0.3059 - val_accuracy: 0.8740
Epoch 4/20
25000/25000 [==============================] - 24s 945us/sample - loss: 0.2193 - accuracy: 0.9142 - val_loss: 0.2940 - val_accuracy: 0.8764
Epoch 5/20
25000/25000 [==============================] - 24s 951us/sample - loss: 0.1719 - accuracy: 0.9382 - val_loss: 0.2910 - val_accuracy: 0.8766
Epoch 6/20
25000/25000 [==============================] - 24s 948us/sample - loss: 0.1346 - accuracy: 0.9541 - val_loss: 0.2934 - val_accuracy: 0.8742
Epoch 7/20
25000/25000 [==============================] - 24s 949us/sample - loss: 0.1029 - accuracy: 0.9

## 초매개변수 튜닝과 AutoML
- **초매개변수 hyperparameter**
  - 주어진 신경망에 대해 최적화할 수 있는 여러 매개변수
  - ex. 은닉 뉴런의 개수, BATCH_SIZE, 에폭 수, 망 자체의 복잡도에 종속된 매개변수 등
  - 신경망 자체의 매개변수(가중치 및 편향 값)와 구별하고자 함
- 초매개변수 튜닝(hyperparameter tuning)
  - 비용 함수를 최소화하는 초매개변수의 최적 조합을 찾는 과정
  - n개의 초매개변수가 이루는 n차원의 공간을 정의, 이 공간에서 비용 함수의 최적 값의 지점 차기
  - 방법: 공간에 그리드를 만들어, 각 그리드 정점에 대한 비용 함수 값 확인 => 초매개변수를 버킷으로 나눠 서로 다른 조합을 무차별 접근 방식으로 확인
- AutoML: 초매개변수를 자동으로 튜닝하고 최적의 신경망 아키텍처를 자동으로 검색하는 것이 목표인 연구 기법

## 출력 예측
- 신경망이 훈련되면 예측에 사용될 수 있다.
```
# 예측하기
predictions = model.predict(X)
```
- 주어진 입력에 대해 출력 계산
  - `model.evaluate()`: 손실 값 계산
  - `model.predict_class()`: 범주 출력 계산
  - `model.predict_proba()`: 부류 확률 계산

## 역전파에 대한 실용적 개괄
- 다층 퍼셉트론은 **역전파 backpropagation** 프로세스로 훈련 데이터에서 학습
- 각 신경망 계층에는 주어진 입력 집합에 대해 출력값을 결정하는 관련 가중치 집합이 있다.
- 신경망은 다수의 은닉층을 가질 수 있다.
- **순전파 propagate forward**
  1. 모든 가중치에 임의의 값 할당
  2. 훈련 집합의 각 입력에 대해 신경망 활성화
    - 값은 입력 단계에서 은닉 단계를 거쳐 출력 단계로 순방향(forward) 전파

![image](https://user-images.githubusercontent.com/61455647/118077076-61d6fd80-b3ee-11eb-82e0-5a6779e96c5c.png)

- 역전파
  - 훈련 집합에서 실제 관측값을 알기 때문에 예측에서 발생한 오차 계산
  - GD처럼 적절한 최적화 알고리즘으로 오차를 줄이려는 목적
  - 신경망 가중치를 조정하고자 오차를 역으로 전파

![image](https://user-images.githubusercontent.com/61455647/118077218-a2cf1200-b3ee-11eb-9010-37f4441bf587.png)

- 순방향 전파와 역방향 전파 프로세스는 오차가 사전 정의된 임계값 이하로 떨어질 때까지 여러 번 반복된다.
  - 특징: 입력
  - 레이블: 학습 과정 진행에 사용
  - 모델: 손실 함수가 점진적으로 최소화되는 방향으로 갱신
  - 신경망은 정확하게 예측된 레이블 수를 증가시키는 방향으로 내부 가중치를 점진적으로 조정

![image](https://user-images.githubusercontent.com/61455647/118077333-dca01880-b3ee-11eb-8c32-61ada962135d.png)

## 정리

## 딥러닝 접근법을 향해
- 필기체 숫자 인식 중 99% 정확도에 가까워질수록 추가 개선이 어렵다는 결론을 얻었다.
- 적용하지 않은 개선 가능한 점: 이미지의 로컬 공간 구조를 활용하지 않았음=기록된 각 숫자를 나타내는 비트맵을 평면 벡터로 변환해 로컬 공간 구조(인접한 픽셀이 서로 더 가까이 있다)가 사라졌다.
- -> CNN(COnvolutional Neural Network)은 특정 유형의 딥러닝 망이 이미지의 로컬 공간 구조를 보존한다+점진적 추상화 레벨을 통한 학습을 이용해 개발